In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
df_ = pd.read_excel('/content/gdrive/MyDrive/DSMLBC10/week_4 (20.10.22-26.10.22)/rfm/online_retail_II.xlsx', sheet_name = "Year 2009-2010")
df = df_.copy()
df.head()
df.isnull().sum()
df = df[~df["Invoice"].str.contains("C", na=False)]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,515255.00000,10.95663,104.35401,-9600.00000,1.00000,3.00000,10.00000,19152.00000
Price,515255.00000,3.95637,127.68856,-53594.36000,1.25000,2.10000,4.21000,25111.09000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


In [ ]:
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [ ]:
cltv_c = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()})

cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']

In [ ]:
cltv_c

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000
...,...,...,...
18283.00000,6,336,641.77000
18284.00000,1,494,461.68000
18285.00000,1,145,427.00000


In [ ]:
cltv_c["fark"]= cltv_c.shape[0] - cltv_c[cltv_c["total_transaction"] > 1].shape[0]  #tüm müşteri - birden fazla alışveriş yapan müşteri sayısı
cltv_c

,total_transaction,total_unit,total_price,fark
Customer ID,,,,
12346.00000,11,70,372.86000,1421
12347.00000,2,828,1323.32000,1421
12348.00000,1,373,222.16000,1421
12349.00000,3,993,2671.14000,1421
12351.00000,1,261,300.93000,1421
...,...,...,...,...
18283.00000,6,336,641.77000,1421
18284.00000,1,494,461.68000,1421
18285.00000,1,145,427.00000,1421


In [ ]:
cltv_c["cltv"] = (cltv_c["total_price"] * cltv_c["total_price"]* 0.10)/(cltv_c["fark"])
cltv_c

,total_transaction,total_unit,total_price,fark,cltv
Customer ID,,,,,
12346.00000,11,70,372.86000,1421,9.78357
12347.00000,2,828,1323.32000,1421,123.23546
12348.00000,1,373,222.16000,1421,3.47326
12349.00000,3,993,2671.14000,1421,502.11041
12351.00000,1,261,300.93000,1421,6.37290
...,...,...,...,...,...
18283.00000,6,336,641.77000,1421,28.98443
18284.00000,1,494,461.68000,1421,14.99989
18285.00000,1,145,427.00000,1421,12.83103


In [ ]:
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,fark,cltv
Customer ID,,,,,
18102.00000,89,124216,349164.35000,1421,8579573.77276
14646.00000,78,170342,248396.50000,1421,4342070.45829
14156.00000,102,108107,196566.74000,1421,2719105.08615
14911.00000,205,69722,152147.57000,1421,1629055.80978
13694.00000,94,125893,131443.19000,1421,1215855.89003


Müşterileri segmentlere ayırarak hangi müşteriye daha fazla önem vereceğini bulabilirsin.

In [ ]:
cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels = ["D", "C", "B", "A"])

In [ ]:
cltv_c.head()

,total_transaction,total_unit,total_price,fark,cltv,segment
Customer ID,,,,,,
12346.00000,11,70,372.86000,1421,9.78357,C
12347.00000,2,828,1323.32000,1421,123.23546,B
12348.00000,1,373,222.16000,1421,3.47326,D
12349.00000,3,993,2671.14000,1421,502.11041,A
12351.00000,1,261,300.93000,1421,6.37290,D


In [ ]:
cltv_c.sort_values(by ="cltv", ascending = False).head()

,total_transaction,total_unit,total_price,fark,cltv,segment
Customer ID,,,,,,
18102.00000,89,124216,349164.35000,1421,8579573.77276,A
14646.00000,78,170342,248396.50000,1421,4342070.45829,A
14156.00000,102,108107,196566.74000,1421,2719105.08615,A
14911.00000,205,69722,152147.57000,1421,1629055.80978,A
13694.00000,94,125893,131443.19000,1421,1215855.89003,A


In [ ]:
cltv_c.groupby("segment").agg({"count","mean","sum"})

total_transaction                 total_unit                      \
                    count    sum     mean      count      sum       mean   
segment                                                                    
D                    1079   1326  1.22892       1079   117616  109.00463   
C                    1078   2160  2.00371       1078   305135  283.05659   
B                    1078   4063  3.76902       1078   733211  680.15863   
A                    1079  11666 10.81186       1079  4383262 4062.33735   

        total_price                           fark                      cltv  \
              count           sum       mean count      sum       mean count   
segment                                                                        
D              1079  192265.13000  178.18826  1079  1533259 1421.00000  1079   
C              1078  513016.45300  475.89652  1078  1531838 1421.00000  1078   
B              1078 1219605.20000 1131.35918  1078  1531838 1421.00000  1078   
A              1079 6907116.49100 6401.40546  1079  1533259 1421.00000  1079   

                                    
                   sum        mean  
segment                             
D           2849.32790     2.64071  
C          18184.26342    16.86852  
B         103549.86329    96.05739  
A       25257295.04118 23408.05843